### Two kinds of averaging in this notebook:
##### 1. Averaging over samples, so for each run, we have average gene scores for each phenotype 
(#runs x #samples x #genes x #phenotypes) --> (#runs x #genes x #phenotypes)

##### 2. Averaging over runs and phenotypes, so for each sample, we have a gene importance score
(#runs x #samples x #genes x #phenotypes) --> (#samples x #genes)


## 1. Average over samples for each run:

In [1]:
import gc
import h5py
import numpy as np
import pandas as pd


import scipy
from scipy import stats 
import datetime 

import h5py

import sys

import pickle

from functools import reduce

import os

from matplotlib import pyplot as plt
%matplotlib inline


path_to_configs = "../"
sys.path.append(path_to_configs)
from configs import * 

/home/nbbwang/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
phenotypes = ["CERAD", "BRAAK", "PLAQUES", "TANGLES", "ABETA_IHC", "TAU_IHC"]

#### Getting high and no pathology groups

In [3]:
path_to_data = path_to_configs + path_to_MDAD_data_folders

num_components=500
with h5py.File(path_to_data + "ACT_MSBBRNA_ROSMAP_PCA.h5", 'r') as hf:
    X = hf["ge_transformed"][:,:num_components].astype(np.float64)      
    Y = hf["labels"][:]
#     X = hf["X"][:,:num_components].astype(np.float64)      
#     Y = hf["Y"][:]
    PCA_components = hf["PCA_components_"][:]
    gene_symbols = hf["gene_symbols"][:]
    labels_names = hf["labels_names"][:]
    
merged_phenotypes = pd.read_csv(path_to_data + "merged_phenotypes.csv")
labels_df = pd.DataFrame(Y.astype(str), columns=labels_names.astype(str))
labels_df = labels_df.merge(merged_phenotypes, how="left", on="sample_name")

resilient_conditions = [np.where(labels_df["BRAAK"].astype(float) == 5)[0],\
 np.where(labels_df["CERAD"].astype(float) > 2)[0], 
 np.where(labels_df["dementia"] == 0)[0]]
resilient_idx = np.sort(list(reduce(set.intersection, [set(item) for item in resilient_conditions])))


resistant_conditions = [np.where(labels_df["BRAAK"].astype(float) <= 2)[0],\
 np.where(labels_df["CERAD"].astype(float) <= 2)[0],\
 np.where(labels_df["dementia"] == 0)[0],\
 np.where(labels_df["age_censored"].values.astype(float) > 85)[0]]
resistant_idx = np.sort(list(reduce(set.intersection, [set(item) for item in resistant_conditions])))


AD_conditions = [np.where(labels_df["BRAAK"].astype(float) == 5)[0],\
 np.where(labels_df["CERAD"].astype(float) > 2)[0],\
 np.where(labels_df["dementia"] == 1)[0]]
AD_idx = np.sort(list(reduce(set.intersection, [set(item) for item in AD_conditions])))


all_idx = np.union1d(np.union1d(resistant_idx, resilient_idx), AD_idx)
# all_idx = np.union1d(AD_idx, resilient_idx)
# all_idx = resilient_idx

color_labels = ["r" if x in AD_idx else("g" if x in resilient_idx else "b") for x in all_idx]

np.array([len(AD_idx) , len(resilient_idx) , len(resistant_idx)])/np.sum([len(AD_idx) , len(resilient_idx) , len(resistant_idx)])

high_path_idx = np.union1d(resilient_idx, AD_idx)
no_path_idx = resistant_idx
print("High path: %i, Low path: %i"%(len(high_path_idx), len(no_path_idx)))

High path: 392, Low path: 120


#### Averaged weights for high pathology (Braak 5or6, CERAD  3) * +1 , no pathology * -1
Note: this is a weighted average where pathology gets a weight of 1, and no pathology gets a weight of -1 -- so this shows which genes have highly positive weights for AD but negative for control, and the opposite

In [13]:
method="MTL"
for rep in range(100): 
    for phenotype in phenotypes:
        print(phenotype)

        IG_path = "%s%s/%s/%i/outputs/%s.h5"%(path_to_configs + IG_save_path, SPECIFIC_FOLDER, method,rep, phenotype)
        print(IG_path)


        with h5py.File(IG_path, 'r') as hf:
            gw  = hf["gene_weights"][:][0]


        ######## Averaged weights for high pathology (Braak 5or6, CERAD  3) * +1 , no pathology * -1

        weighted_gw = np.vstack([gw[high_path_idx], -1*gw[no_path_idx]])
        avged = np.mean(weighted_gw,axis=0)

        savefolder_name = "weighted_avg_high_vs_low_path"
        newsavepath = "%s%s/AVERAGING/%s/%s/%i/outputs/"%(path_to_configs+IG_save_path, SPECIFIC_FOLDER, savefolder_name, method, rep)
        if not os.path.isdir(newsavepath):
            os.makedirs(newsavepath)   

        with h5py.File(newsavepath + "%s.h5"%phenotype, 'w') as hf:
            hf.create_dataset("gene_weights", data=avged) 
        print("saved to %s"%(newsavepath + "%s.h5"%phenotype))


# 2.  Getting consensus gene importance scores for each sample:

In [18]:
phenotypes = ["CERAD", "PLAQUES", "ABETA_IHC", "BRAAK", "TANGLES", "TAU_IHC"]

consensus_gws = np.zeros([len(phenotypes),len(X), len(gene_symbols)])
    
for run in range(100):
    print(run)
    for i,phen in enumerate(phenotypes):
        path = "origGE/MTL/%s/outputs/%s.h5"%(path_to_configs + IG_save_path, run,phen)
        with h5py.File(path, 'r') as hf:
            gw = hf["gene_weights"][:][0]
        consensus_gws[i] += gw

In [19]:
gene_scores_for_samples = np.mean(consensus_gws/100,axis=0)
path_to_save_scores = path_to_configs + path_to_gene_rankings + "MTL/"

np.savetxt(path_to_save_scores + "gene_scores_for_samples.txt", gene_scores_for_samples)

In [12]:
path_to_save_scores + "gene_scores_for_samples.txt"

'../../../Pipeline_Outputs_Submitted/gene_rankings/MTL/gene_scores_for_samples.txt'